In [1]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

In [2]:
openai_api_key = os.environ["OPENAI_API_KEY"]

# Completion Model

In [3]:
from langchain_openai import OpenAI
openai_llm_model = OpenAI()

# Chat Model

In [4]:
from langchain_openai import ChatOpenAI

openai_chat_llm_model = ChatOpenAI(model="gpt-3.5-turbo-0125")

## Prompts and Prompt Templates

In [5]:
# for completion model
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    "Tell me a {adjective} story about {topic}."
)

# Pass the input to the prompt (any input that matches the template)
llmModelPrompt = prompt_template.format(
    adjective="curious",
    topic="The Herzog von Sachsen king Family"
)

# Call the completion model
res = openai_llm_model.invoke(llmModelPrompt)
print(res)



The Herzog von Sachsen family was renowned for their lavish lifestyle and extravagant parties. However, there was one member of the family who was notoriously known for his eccentric behavior – King Friedrich August III.

King Friedrich August III was known as the "Mad King" among the people of Sachsen due to his unpredictable actions and odd habits. He had a peculiar fascination with clocks and would spend hours tinkering with them in his palace. But what made him truly unique was his love for animals.

The king had a private zoo in his palace, filled with exotic animals from all around the world. However, his favorite animal was a chimpanzee named Hans, whom he had raised since birth. Hans was treated like a member of the royal family, with a personal servant to cater to his every need.

But what made Hans truly special was his ability to mimic human behavior. The king had taught him how to walk, talk, and even dance. Hans became the talk of the town, and people would often come to

In [6]:
# for chat completion model
from langchain_core.prompts import ChatPromptTemplate

# Create a chat template
chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are an {profession} expert on {topic}."),
        ("user", "Hello, Mr. {profession} expert, can you please answer a question?"),
        ("ai", "Sure! What is your question?"),
        ("user", "{user_input}")
    ]
)

# Pass the input to the prompt (any input that matches the template) (Few-Shot Prompting)
messages = chat_template.format_messages(
    profession="IT",
    topic="Artificial Intelligence",
    user_input="Are the knowledge of NLP & Deep Learning required to learn Generative AI?"
)

# Call the chat completion model
response = openai_chat_llm_model.invoke(messages)
print(response.content)

Yes, knowledge of Natural Language Processing (NLP) and Deep Learning are beneficial when learning about Generative AI. Generative AI involves creating models that can generate new, realistic data based on patterns learned from existing data. NLP and Deep Learning techniques are often used in Generative AI to generate text, images, music, and more. Understanding NLP can help with text generation tasks, while knowledge of Deep Learning can help with building and training complex generative models such as Generative Adversarial Networks (GANs) and Variational Autoencoders (VAEs). Having a solid foundation in NLP and Deep Learning concepts will definitely be advantageous when diving into the field of Generative AI.


## Few Shot Prompting

In [7]:
from langchain_core.prompts import FewShotChatMessagePromptTemplate

# Englsih to Spanish translation example
# This example is a few-shot prompt for translating English to Spanish.
# The example consists of a list of dictionaries, each containing an input and output pair.
# The input is an English phrase, and the output is its Spanish translation.
examples = [
    {"input": "hi!", "output": "hola!"},
    {"input": "bye!", "output": "adios!"}
]

# Create a few-shot prompt template
# The example prompt is defined using the ChatPromptTemplate class.
# It consists of a user message with the input and an AI message with the output.
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("user", "{input}"),
        ("ai", "{output}")
    ]
)

# The example prompt is then used to create a FewShotChatMessagePromptTemplate.
# This template takes the example prompt and the list of examples to create a few-shot prompt.
# The example prompt is used to format the examples into a few-shot format.
# The examples are passed to the example prompt to create a formatted example.
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples
)


# The final prompt is created by combining the system message, the few-shot prompt, and the user input.
# The system message instructs the model to act as an English-Spanish translator.
# The user input is passed to the final prompt for translation.
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are an English-Spanish translator."),
        few_shot_prompt,
        ("user", "{input}")
    ]
)

## Chains

In [ ]:
examples = [
    {"input": "hi!", "output": "hola!"},
    {"input": "bye!", "output": "adios!"}
]

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("user", "{input}"),
        ("ai", "{output}")
    ]
)

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are an English-Spanish translator."),
        few_shot_prompt,
        ("user", "{input}")
    ]
)

# The final prompt is then used to create a chain with the chat model.
# The chain takes the final prompt and the chat model as input.
chain = final_prompt | openai_chat_llm_model # LangChain Expression Language (LCL) syntax

# The chain is then used to invoke the model with the user input.
res = chain.invoke({"input": "How are you?"})
print(res.content)

¿Cómo estás?


https://python.langchain.com/docs/introduction/

https://python.langchain.com/docs/concepts/lcel/

## Output Parsers

* Used to reformat the response of the large language model.
* We can reformat the text in any kind of format, whether it's a json format or txt format.

In [9]:
from langchain_core.prompts import PromptTemplate
from langchain.output_parsers.json import SimpleJsonOutputParser

json_prompt = PromptTemplate.from_template(
    "Return a JSON object with an `answer` key that answers the following question: {question}"
)

json_parser = SimpleJsonOutputParser()

# The JSON chain is created by combining the prompt, the model, and the output parser.
# The chain takes the JSON prompt, the OpenAI model, and the JSON parser as input.
json_chain = json_prompt | openai_llm_model | json_parser

# The chain is then used to invoke the model with the user input.
# The input is a question, and the output is a JSON object with the answer.
res = json_chain.invoke({"question": "What is the biggest country in the world?"})
print(res)

{'answer': 'Russia'}


In [10]:
type(res)

dict

In [13]:
json_parser.get_format_instructions()

'Return a JSON object.'

### Optionally, you can use Pydantic to define a custom output format.

In [ ]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field

In [12]:
# Define a Pydantic Object with the desired output format
class Joke(BaseModel):
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")

In [14]:
# Define the parser referring the Pydantic Object
parser = JsonOutputParser(pydantic_object=Joke)

# Add the parser format instructions to the prompt definition
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

# Create a chain with the prompt and the parser
chain = prompt | openai_chat_llm_model | parser

res = chain.invoke({"query": "Tell me a joke."})
print(res)

{'setup': "Why don't scientists trust atoms?", 'punchline': 'Because they make up everything!'}
